## Train

In [1]:
import numpy as np
import pandas as pd
import keras
import cv2
from keras.applications.efficientnet import EfficientNetB7
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
DATA_DIR = "../data/dog-breed-identification"

# 훈련할 csv 데이터 셋팅
df_train = pd.read_csv(DATA_DIR+"/labels.csv")

# breed 파싱(중첩제거?)
targets_series = pd.Series(df_train['breed'])
# one_hot 변환(0 0 1 0)
one_hot = pd.get_dummies(targets_series, sparse = True)

# 배열로 변환
one_hot_labels = np.asarray(one_hot)

# 이미지 사이즈 정의
im_size = 90

x_train = []
y_train = []

# 훈련할 데이터 셋팅
i = 0
for f, breed in tqdm(df_train.values):
    img = cv2.imread(DATA_DIR+"/train/{}.jpg".format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1


y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

# shape 확인
print(x_train_raw.shape)
print(y_train_raw.shape)

# 분류
num_class = y_train_raw.shape[1]

# 교육할 데이터셋팅 및 성능테스트 데이터 교육데이터에서 30% 할당
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)

100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [00:15<00:00, 655.89it/s]


(10222, 90, 90, 3)
(10222, 120)


In [3]:
# 사전 교육된 기본 모델 생성
base_model = EfficientNetB7(
    weights = 'imagenet',
    # weights = None,
    include_top=False, input_shape=(im_size, im_size, 3))

# model out 정의
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 저장된 weights 로드
# model.load_weights('weights.h5')

# 교육할 계층 설정 최상위 계층만 교육 (False 가중치 고정)
for layer in base_model.layers:
    layer.trainable = False

# 훈련할 레이어 확인
for layer in base_model.layers:
    print(layer, layer.trainable)

# 오차역전파 정의
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
EPOCH = 500
filename = 'EfficientNetB7/EfficientNetB7_DOG_weights-e{}.h5'.format(EPOCH)
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1),
                 keras.callbacks.ModelCheckpoint(filename, monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')]
# model.summary()

Metal device set to: Apple M1 Pro


2022-09-19 03:38:58.327956: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-19 03:38:58.328072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<keras.engine.input_layer.InputLayer object at 0x107833310> False
<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x166415a00> False
<keras.layers.preprocessing.normalization.Normalization object at 0x166415fa0> False
<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x166419b20> False
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x16616b5e0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x16641c820> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x16641c700> False
<keras.layers.core.activation.Activation object at 0x16aa1c2e0> False
<keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x16aaed6d0> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x16aad83d0> False
<keras.layers.core.activation.Activation object at 0x16aad83a0> False
<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x1777b8b80> False
<

In [4]:
from datetime import datetime
from keras.callbacks import EarlyStopping
    
# 학습시작
print("train start  : ", datetime.now())
model.fit(X_train, Y_train, epochs=EPOCH, validation_data=(X_valid, Y_valid), verbose=1, callbacks = callbacks_list)
# model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid), verbose=1)
print("train finish : ", datetime.now())

train start  :  2022-09-19 03:39:02.265602
Epoch 1/500


2022-09-19 03:39:02.635092: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-19 03:39:07.892047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


224/224 [==============================] - ETA: 0s - loss: 20.4195 - accuracy: 0.0089

2022-09-19 03:39:49.501352: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: accuracy improved from -inf to 0.00894, saving model to EfficientNetB7/EfficientNetB7_DOG_weights-e500.h5
224/224 [==============================] - 69s 269ms/step - loss: 20.4195 - accuracy: 0.0089 - val_loss: 14.2399 - val_accuracy: 0.0072
Epoch 2/500
224/224 [==============================] - ETA: 0s - loss: 14.2331 - accuracy: 0.0082
Epoch 2: accuracy did not improve from 0.00894
224/224 [==============================] - 49s 218ms/step - loss: 14.2331 - accuracy: 0.0082 - val_loss: 11.0158 - val_accuracy: 0.0091
Epoch 3/500
224/224 [==============================] - ETA: 0s - loss: 16.5590 - accuracy: 0.0089
Epoch 3: accuracy did not improve from 0.00894
224/224 [==============================] - 48s 216ms/step - loss: 16.5590 - accuracy: 0.0089 - val_loss: 16.0663 - val_accuracy: 0.0088
Epoch 4/500
224/224 [==============================] - ETA: 0s - loss: 15.3490 - accuracy: 0.0089
Epoch 4: accuracy did not improve from 0.00894
224/224 [==============================] 

In [5]:
# 학습된 weights 저장
# model.save('EfficientNetB7_DOG_weights.h5')
model.save_weights('EfficientNetB7_DOG_weights.h5')